In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("terencicp/e-commerce-dataset-by-olist-as-an-sqlite-database")

print("Path to dataset files:", path)

c:\Users\Hosni\miniconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 48.7M/48.7M [00:05<00:00, 8.69MB/s]

Extracting files...


Path to dataset files: C:\Users\Hosni\.cache\kagglehub\datasets\terencicp\e-commerce-dataset-by-olist-as-an-sqlite-database\versions\1


In [52]:
import kagglehub
import sqlite3
import pandas as pd

def load_data(kaggle_path, file_name):
    """
    Downloads the Olist e-commerce dataset from KaggleHub, connects to the SQLite database,
    retrieves table names, and loads data from specific tables into Pandas DataFrames.

    Returns:
        dict: A dictionary containing table names as keys and their corresponding DataFrames as values.
    """
    # Download the SQLite database
    path = kagglehub.dataset_download(kaggle_path)

    # Connect to the SQLite database
    sqlite_file = f"{path}/{file_name}"  # Update with the correct filename if needed
    conn = sqlite3.connect(sqlite_file)

    try:
        # List all tables in the database
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        tables = pd.read_sql(query, conn)
        print("Available tables:", tables)

        table_names = tables.iloc[:, 0].values
        # Load data from each table into a dictionary of DataFrames
        dataframes = {}
        for table in table_names:
            globals()[table] = pd.read_sql(f"SELECT * FROM {table};", conn)
            dataframes[table] = globals()[table]  # Store DataFrame in the dictionary
        
        print(f"Created the {len(table_names)} following dataframes: {table_names}")
    finally:
        # Close the connection
        conn.close()
    return dataframes

# Example usage:
dataframes = load_data(kaggle_path="terencicp/e-commerce-dataset-by-olist-as-an-sqlite-database", file_name="olist.sqlite")


Available tables:                                  name
0   product_category_name_translation
1                             sellers
2                           customers
3                         geolocation
4                         order_items
5                      order_payments
6                       order_reviews
7                              orders
8                            products
9                     leads_qualified
10                       leads_closed
Created the 11 following dataframes: ['product_category_name_translation' 'sellers' 'customers' 'geolocation'
 'order_items' 'order_payments' 'order_reviews' 'orders' 'products'
 'leads_qualified' 'leads_closed']


{'product_category_name_translation':             product_category_name product_category_name_english
 0                    beleza_saude                 health_beauty
 1          informatica_acessorios         computers_accessories
 2                      automotivo                          auto
 3                 cama_mesa_banho                bed_bath_table
 4                moveis_decoracao               furniture_decor
 ..                            ...                           ...
 66                         flores                       flowers
 67             artes_e_artesanato         arts_and_craftmanship
 68                fraldas_higiene           diapers_and_hygiene
 69  fashion_roupa_infanto_juvenil     fashion_childrens_clothes
 70             seguros_e_servicos         security_and_services
 
 [71 rows x 2 columns],
 'sellers':                              seller_id  seller_zip_code_prefix  \
 0     3442f8959a84dea7ee197c632cb2df15                   13023   
 1     d1b65

In [58]:
def dataset_exploration(df, df_name):
    total_nulls = df.isnull().sum().sum()
    num_duplicated = df.duplicated().sum()

    print(f"Look into {df_name} table:")
    print(f"\nThis dataframe contains {df.shape[0]} rows and {df.shape[1]} columns.")
    print(f"\nThis dataframe contains the following columns: {df.columns.tolist()}")
    print(f"\nThis dataframe contains {total_nulls} null values.")
    print(f"\nThis dataframe contains {num_duplicated} duplicated rows.")
    print("_"*30)


In [61]:
for name, table in dataframes.items():
    dataset_exploration(table, name)

Look into product_category_name_translation table:

This dataframe contains 71 rows and 2 columns.

This dataframe contains the following columns: ['product_category_name', 'product_category_name_english']

This dataframe contains 0 null values.

This dataframe contains 0 duplicated rows.
______________________________
Look into sellers table:

This dataframe contains 3095 rows and 4 columns.

This dataframe contains the following columns: ['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']

This dataframe contains 0 null values.

This dataframe contains 0 duplicated rows.
______________________________
Look into customers table:

This dataframe contains 99441 rows and 5 columns.

This dataframe contains the following columns: ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']

This dataframe contains 0 null values.

This dataframe contains 0 duplicated rows.
______________________________
Look into geolocation table:

#### We won't be using leads_closed and leads_qualified tables so we won't be looking into them for the next steps.
#### The following needs to be looked at:
- 261831 duplicated rows on the geolocation table.
- 145903 null values on the order_reviews table.
- 4908 null values on the orders table.
- 2448 null values on the products table.

### Table Cleaning: Geolocation

In [63]:
duplicated_geolocation = geolocation[geolocation.duplicated()]
print(f"\nDuplicated rows in geolocation table:")
duplicated_geolocation


Duplicated rows in geolocation table:


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
15,1046,-23.546081,-46.644820,sao paulo,SP
44,1046,-23.546081,-46.644820,sao paulo,SP
65,1046,-23.546081,-46.644820,sao paulo,SP
66,1009,-23.546935,-46.636588,sao paulo,SP
67,1046,-23.546081,-46.644820,sao paulo,SP
...,...,...,...,...,...
1000153,99970,-28.343273,-51.873734,ciriaco,RS
1000154,99950,-28.070493,-52.011342,tapejara,RS
1000159,99900,-27.877125,-52.224882,getulio vargas,RS
1000160,99950,-28.071855,-52.014716,tapejara,RS


In [64]:
### Removing Duplicates
geolocation_cleaned = geolocation.drop_duplicates()
print(f"Removed {geolocation.shape[0] - geolocation_cleaned.shape[0]} duplicated rows.")

Removed 261831 duplicated rows.


_______________________________________

### Table Cleaning: Order Reviews

In [65]:
null_rows_order_reviews = order_reviews[order_reviews.isnull().any(axis=1)]
print(f"\nRows with null values in order_reviews table:")
null_rows_order_reviews


Rows with null values in order_reviews table:


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,None,None,2018-07-07 00:00:00,2018-07-14 17:18:30
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,None,None,2017-12-09 00:00:00,2017-12-11 20:06:42
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,None,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,None,None,2018-07-01 00:00:00,2018-07-02 12:59:13


##### The null values are mainly situated in the title and the message, which is totally fine, since those are values we won't be needing in the future unless we are going to do NLP which we are not. -> We drop the columns.

In [67]:
### Dropping Irrelevant Columns
order_reviews_cleaned = order_reviews.drop(columns=['review_comment_title', 'review_comment_message'])
print(f"{order_reviews_cleaned.isnull().sum().sum()} null values remaining.")

0 null values remaining.


_______________________________________________

### Table Cleaning: Orders 

In [70]:
null_rows_orders = orders[orders.isnull().any(axis=1)]
print(f"\nRows with null values in orders table:")
null_rows_orders


Rows with null values in orders table:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,None,None,2017-05-09 00:00:00
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,None,2018-06-28 00:00:00
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,None,None,2018-08-21 00:00:00
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,None,None,2017-10-03 00:00:00
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,None,2018-02-07 00:00:00
...,...,...,...,...,...,...,...,...
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,None,None,None,2018-10-01 00:00:00
99313,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,2018-01-09 07:18:05,None,None,2018-02-06 00:00:00
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,None,None,None,2018-09-27 00:00:00
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,None,None,2017-09-15 00:00:00


In [72]:
### Dropping Irrelevant Columns
orders_cleaned = orders.drop(columns=['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date'])
print(f"{orders_cleaned.isnull().sum().sum()} null values remaining.")

0 null values remaining.


______________________________________________________

### Table Cleaning: Product

In [73]:
null_rows_products = products[products.isnull().any(axis=1)]
print(f"\nRows with null values in order_reviews table:")
null_rows_products


Rows with null values in order_reviews table:


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
105,a41e356c76fab66334f36de622ecbd3a,None,NaN,NaN,NaN,650.0,17.0,14.0,12.0
128,d8dee61c2034d6d075997acef1870e9b,None,NaN,NaN,NaN,300.0,16.0,7.0,20.0
145,56139431d72cd51f19eb9f7dae4d1617,None,NaN,NaN,NaN,200.0,20.0,20.0,20.0
154,46b48281eb6d663ced748f324108c733,None,NaN,NaN,NaN,18500.0,41.0,30.0,41.0
197,5fb61f482620cb672f5e586bb132eae9,None,NaN,NaN,NaN,300.0,35.0,7.0,12.0
...,...,...,...,...,...,...,...,...,...
32515,b0a0c5dd78e644373b199380612c350a,None,NaN,NaN,NaN,1800.0,30.0,20.0,70.0
32589,10dbe0fbaa2c505123c17fdc34a63c56,None,NaN,NaN,NaN,800.0,30.0,10.0,23.0
32616,bd2ada37b58ae94cc838b9c0569fecd8,None,NaN,NaN,NaN,200.0,21.0,8.0,16.0
32772,fa51e914046aab32764c41356b9d4ea4,None,NaN,NaN,NaN,1300.0,45.0,16.0,45.0


##### Products with no category or description whatsoever are not relevant to our analysis since they offer no insight.

In [77]:
### Dropping Null values
products_cleaned = products.dropna(subset=['product_category_name'])
print(f"{products_cleaned.isnull().sum().sum()} null values remaining.")

4 null values remaining.


In [80]:
null_rows_products = products_cleaned[products_cleaned.isnull().any(axis=1)]
null_rows_products

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8578,09ff539a621711667c43eba6a3bd8466,bebes,60.0,865.0,3.0,NaN,NaN,NaN,NaN


### The data is now ready to be worked on.

In [88]:
# Exporting the data to CSV files
seller.to_csv('data_output/seller.csv', index=False)
products_cleaned.to_csv('data_output/products.csv', index=False)
orders_cleaned.to_csv('data_output/orders.csv', index=False)
order_reviews_cleaned.to_csv('data_output/order_reviews.csv', index=False)
geolocation_cleaned.to_csv('data_output/geolocation.csv', index=False)
order_items.to_csv('data_output/order_items.csv', index=False)
order_payments.to_csv('data_output/order_payments.csv', index=False)
customers.to_csv('data_output/customers.csv', index=False)

In [84]:
table_names

array(['product_category_name_translation', 'sellers', 'customers',
       'geolocation', 'order_items', 'order_payments', 'order_reviews',
       'orders', 'products', 'leads_qualified', 'leads_closed'],
      dtype=object)